In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')


service = Service(ChromeDriverManager().install())

In [10]:
def extraer_datos_bgg(driver, esperar_carga=True, delay=5):
    """
    Extrae y muestra datos de juegos de mesa de BGG.
    """
    if esperar_carga:
        print(f"Esperando {delay} segundos para que cargue la página...")
        time.sleep(delay)

    # Encontrar todas las filas con id que contiene "row_"
    # Opción 1: Usando XPath para encontrar elementos tr con id que contiene "row_"
    filas = driver.find_elements(By.XPATH, "//tr[contains(@id, 'row_')]")

    # Opción 2: Alternativa usando CSS selector
    # filas = driver.find_elements(By.CSS_SELECTOR, "tr[id^='row_']")

    if not filas:
        print("No se encontraron filas en la página.")
        return

    print(f"Se encontraron {len(filas)} filas de juegos:")
    print("-" * 50)

    for i, fila in enumerate(filas, start=1):
        try:
            # Extraer los datos de cada columna
            rank = fila.find_element(By.CSS_SELECTOR, "td.collection_rank").text
            titulo = fila.find_element(By.CSS_SELECTOR, "td.collection_objectname").text
            geek_rating = fila.find_element(By.CSS_SELECTOR, "td.collection_bggrating").text

            print(f"Juego {i}:")
            print(f" Rank: {rank}")
            print(f" Título: {titulo}")
            print(f" Geek Rating: {geek_rating}")
            print("-" * 50)
        except Exception as e:
            print(f"Error al procesar fila {i}: {e}")

In [11]:
# Inicializar el webdriver
driver = webdriver.Chrome(options=chrome_options)

# Navegar a la página de YouTube
driver.get('https://boardgamegeek.com/geeksearch.php?action=search&q=ra&objecttype=boardgame')

# Llamar a la función para extraer videos
extraer_datos_bgg(driver, esperar_carga=False)

# Cerrar el navegador
driver.quit()

Se encontraron 100 filas de juegos:
--------------------------------------------------
Juego 1:
 Rank: 131
 Título: Ra (1999)
Bid to acquire the most valuable sets of Egyptian artifacts and resources.
 Geek Rating: 7.501
--------------------------------------------------
Juego 2:
 Rank: 2105
 Título: Ra: The Dice Game (2009)
Roll dice to build an Egyptian civilization over three eras.
 Geek Rating: 6.202
--------------------------------------------------
Juego 3:
 Rank: N/A
 Título: Ra: Strategy Among Hexagons (1981)
 Geek Rating: N/A
--------------------------------------------------
Juego 4:
 Rank: N/A
 Título: Ra: Traders (2025)
Expand your Dynasty through trader ships on the Nile
 Geek Rating: N/A
--------------------------------------------------
Juego 5:
 Rank: N/A
 Título: RA and Write (2025)
Make your mark on Egyptian history!
 Geek Rating: N/A
--------------------------------------------------
Juego 6:
 Rank: N/A
 Título: Ra Ra Tuin
Make the right question to guess the image.


In [12]:
def visitar_articulos(driver, esperar_carga=True, delay=3):
    """
    Visita cada artículo de juego de mesa encontrado en la página de resultados
    y permite extraer información detallada de cada uno.

    Args:
        driver: Instancia del navegador
        esperar_carga: Si debe esperar a que cargue la página
        delay: Tiempo de espera en segundos
    """
    # Encontrar todos los enlaces a artículos
    filas = driver.find_elements(By.XPATH, "//tr[contains(@id, 'row_')]")

    if not filas:
        print("No se encontraron artículos para visitar.")
        return

    print(f"Visitando {len(filas)} artículos...")

    for i, fila in enumerate(filas, start=1):
        try:
            # Encontrar el enlace dentro de la columna del título
            enlace = fila.find_element(By.CSS_SELECTOR, "td.collection_objectname a")
            titulo = enlace.text
            url = enlace.get_attribute('href')

            print(f"Visitando artículo {i}: {titulo}")

            # Guardar la ventana actual
            ventana_principal = driver.current_window_handle

            # Abrir el enlace en una nueva pestaña
            driver.execute_script("window.open(arguments[0]);", url)

            # Cambiar a la nueva pestaña
            driver.switch_to.window(driver.window_handles[1])

            if esperar_carga:
                time.sleep(delay)

            # Aquí puedes llamar a otra función para extraer datos detallados
            extraer_datos_detallados(driver, titulo)

            # Cerrar la pestaña y volver a la principal
            driver.close()
            driver.switch_to.window(ventana_principal)

        except Exception as e:
            print(f"Error al visitar artículo {i}: {e}")
            # Asegurarse de volver a la ventana principal si hay un error
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

def extraer_datos_detallados(driver, titulo):
    """
    Extrae información detallada de la página de un juego de mesa.

    Args:
        driver: Instancia del navegador
        titulo: Título del juego para referencia
    """
    try:
        # Ejemplo de extracción de datos detallados - ajusta según la estructura de la página
        descripcion = driver.find_element(By.XPATH, "//div[contains(@class, 'game-description')]").text
        rating = driver.find_element(By.XPATH, "//div[contains(@class, 'game-rating')]").text

        print(f" - Detalles de '{titulo}':")
        print(f"   Descripción: {descripcion[:100]}...")
        print(f"   Rating: {rating}")

        # Añade más extracciones según necesites

    except Exception as e:
        print(f" - Error al extraer datos detallados: {e}")

Ahora vamos a scrapear individualmente

In [ ]:
url='https://boardgamegeek.com/boardgame/12/ra'